# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = get_date_list(data)
data = read_data(data)

Record num : 5429


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


    # lema before after loops --------------------------------
    if len(data['lema_list']) < data['lema_len']:
        data = before_lema(data)        
        continue

    if len(data['lema_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = get_trend_lines(data)
    data = trend_angle_check(data)

100%|█████████████████████████████████████| 5429/5429 [00:28<00:00, 191.73it/s]


In [4]:
data['df']

,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h,l,h_line,l_line
0,20210105 00:00:00.070,1.22525,1.22531,1.225280,2021-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210105 00:00:02.487,1.22525,1.22530,1.225275,2021-01-05 00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210105 00:00:03.611,1.22526,1.22528,1.225270,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210105 00:00:03.662,1.22522,1.22527,1.225245,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210105 00:00:03.764,1.22521,1.22526,1.225235,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5424,20210105 01:59:31.385,1.22569,1.22570,1.225695,2021-01-05 01:59:31,1.225804,1.225906,down,,NaN,-17.108497,-9.933569,NaN,NaN,NaN,1.225876,1.225717
5425,20210105 01:59:31.487,1.22568,1.22570,1.225690,2021-01-05 01:59:31,1.225802,1.225905,down,,NaN,-17.108497,-9.933569,NaN,NaN,NaN,1.225876,1.225717
5426,20210105 01:59:31.998,1.22567,1.22570,1.225685,2021-01-05 01:59:31,1.225800,1.225905,down,,NaN,-17.108497,-9.933569,NaN,NaN,NaN,1.225875,1.225717
5427,20210105 01:59:32.165,1.22566,1.22570,1.225680,2021-01-05 01:59:32,1.225798,1.225904,down,,NaN,-17.108497,-9.933569,NaN,NaN,NaN,1.225875,1.225716


In [5]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 5429
Hours     : 1.99
Recs/hour : 2728
Recs/min  : 45
Recs/sec  : 0.8


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h,l,h_line,l_line
0,20210105 00:00:00.070,1.22525,1.22531,1.225280,2021-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210105 00:00:02.487,1.22525,1.22530,1.225275,2021-01-05 00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210105 00:00:03.611,1.22526,1.22528,1.225270,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210105 00:00:03.662,1.22522,1.22527,1.225245,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210105 00:00:03.764,1.22521,1.22526,1.225235,2021-01-05 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)